In [1]:
from false_positives_negatives import *
%matplotlib widget

# H. Pylori
*Helicobacter Pylori* is a type of bacteria that can cause infections in the stomach. Symptoms of *H. pylori* include painful stomach ulcers. A common test for the infection is the urea breath test. After the patient ingests a tablet or drink containing urea enriched with the isotope carbon-13 (rather than the usual carbon-12), the test measures the proportion of carbon-13 in the carbon dioxide in the collected breath sample. If the amount of carbon-13 is in excess of the normal amount of carbon-13 in one's breath, then it suggests the presence of the bacteria in the stomach, because the bacteria have converted the ingested urea into carbon dioxide. The test result is how much excess carbon-13 is found in one's breath. For example, a test result of 5 means that there is 5 times as much carbon-13 as usual.

As study was done to find the distribution of the urea test results of subjects with or without *H. pylori* infection. Below is a histogram that shows the number of patients with a particular test result. Patients without the infection are plotted in blue; those who are infected are plotted in orange.

https://onlinelibrary.wiley.com/doi/epdf/10.1046/j.1365-2036.2003.01417.x

In [2]:
config_hpylori = {
    'title': 'H. pylori breath test',
    'mean_neg': 0.16,
    'mean_pos': 30.2,
    'std_neg': 0.36,
    'std_pos': 2.5,
    'p_pos': .549,
    'label_neg': 'No H. pylori infection',
    'label_pos': 'H. pylori infected'
}
test_hpylori = Test(config_hpylori)
test_hpylori.generate(1000)
test_hpylori.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=18.822369089759523, description='t', max=40.585787443545215, min=-2.94…

Try slowly moving the slider from side to side. This sets the "threshold" value that delineates positive and negative diagnoses. That is, if a patient's urea test result is above this value, they will be told they are "positive"; if the patient's result is below this value, they will be told they are "negative". As you move the slider, pay attention to the number of true/false positives/negatives.

You are the designer of this test. Your job is to decide on a value for this threshold, which will be used by all the clinical laboratories to make diagnoses. **What threshold value do you choose?**

Answer here

---
In most situations, the delineation between positives and negatives are not that clear. In other words, the test results of positives may often over lap with those of negatives. Let us see such a case.

# Pregnancy Test

In the first days of pregnancy, the body produces a hormone called human chorionic gonadotropin (hCG). A home pregnancy kit works by detecting the presence of this hormone in the urine as an indication of pregnancy. The concentration of hCG is measured in units of milli-international units per millilitre (mIU/mL). hCG levels in a non-pregnant woman is typically less than 5 mIU/mL, and the number increases rapidly after the implantation of the embryo.

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4119102/

http://www.clinlabnavigator.com/human-chorionic-gonadotropin-pregnancy-test.html

In [6]:
config_preg_day9 = {
    'title': 'hCG Levels in Urine on 9-th Day of Pregnancy',
    'mean_neg': 0.1,
    'mean_pos': 4.04,
    'std_neg': 0.02,
    'std_pos': 1.9,
    'floor': 0,
    'p_pos': 0.8,
    'label_neg': 'Not pregnant',
    'label_pos': 'Pregnant'
}
test_preg_day9 = Test(config_preg_day9)
test_preg_day9.generate(1000)
test_preg_day9.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=5.191253936449212, description='t', max=11.124115578105455, min=-0.529…

Home pregnancy test kits can only detect hCG if it is above a certain level. This is called the "sensitivity". If the test returns positive for all results above the sensitivity level and negative otherwise, **what should the sensitivity level of a home pregnancy test be in order to ensure (1) no false positives and (2) less than 10% false negatives on the 9-th day of pregnancy?** Try using the slider again to adjust the sensitivity/threshold and observe the changing numbers below the graph.

In [14]:
config_preg_day11 = {
    'title': 'hCG Levels in Urine on 9-th Day of Pregnancy',
    'mean_neg': 0.1,
    'mean_pos': 25.04,
    'std_neg': 0.02,
    'std_pos': 8.1,
    'floor': 0,
    'p_pos': 0.8,
    'label_neg': 'Not pregnant',
    'label_pos': 'Pregnant'
}
test_preg_day11 = Test(config_preg_day11)
test_preg_day11.generate(1000)
test_preg_day11.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=24.47316570567212, description='t', max=51.35615087515477, min=-2.4098…

__What should the sensitivity level of a home pregnancy test be in order to ensure (1) no false positives and (2) less than 10% false negatives on the 11-th day of pregnancy?__

# Carbon monoxide detector



In [25]:
config_CO_detector = {
    'title': 'Optical Density (%/m)',
    'mean_neg': 0.2,
    'mean_pos': 3,
    'std_neg': 0.1,
    'std_pos': 1.5,
    'floor': 0,
    'p_pos': 0.6,
    'label_neg': 'No Fire',
    'label_pos': 'Fire'
}
test_CO_detector = Test(config_CO_detector)
test_CO_detector.generate(1000)
test_CO_detector.plot()

/Users/winston/Documents/Coding/Jupyter/SSS/false_positives_negatives.py:72: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(6,4))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=3.3445184050306827, description='t', max=7.023488650564435, min=-0.334…

Ask consequences of false positive/negative of smoke alarm. Ask where threshold should be. Why are smoke alarms often annoying?

# SAT scores vs graduating

# Spam vs ham or elicit content censorship (add question on realist vs conventionalist definition)

* implement different scenarios with different distributions
* base rate neglect by varying p_pos and asking how likely they are to have the disease given a positive diagnosis